<a href="https://colab.research.google.com/github/mafux777/Alation_Article/blob/master/Alation_API_Training_November_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alation API Training Worksheet

We need to import just a small number of libraries to create an Alation Instance. You can use this Class to test against any official or unofficial API.

In [ ]:
import pandas as pd
import os
import requests

import time
import json

import pprint
pp = pprint.PrettyPrinter(indent=4)

The class AlationInstance is created with a URL, username and password.

In [ ]:
import urllib

# The AlationInstance class is a handle to an Alation server defined by a URL
# A server admin user name and password needs to be provided and all API actions
# will be run as that user
class AlationInstance():
    # The __init__ method is the constructor used for instantiating
    # email: the up to 30 chars user name, often the email, but for long emails could be cut off
    # password: could be the LDAP password, as well
    # verify: Requests verifies SSL certificates for HTTPS requests, just like a web browser.
    # By default, SSL verification is enabled, and Requests will throw a SSLError if it’s unable to verify the certificate
    def __init__(self, host, account, password, verify=True):
        self.host = host
        self.verify = verify
        self.account = account
        self.password = password
        self.token = self.get_token()
        self.headers = self.login(account, password)

    def get_headers(self):
      return self.headers

    # The login method is used to obtain a session ID and relevant cookies
    # They are cached in the headers variable
    # account: the up to 30 chars user name, often the email, but for long emails could be cut off
    # password: could be the LDAP password, as well
    def login(self, account, password):
        URL = self.host + '/login/'

        s = requests.Session()
        s.get(URL, verify=self.verify)

        # get the cookie token
        csrftoken = s.cookies.get('csrftoken')

        # login with user name and password (and token)
        payload = {"csrfmiddlewaretoken": csrftoken, "ldap_user": account, "password": password}
        headers = {"Referer": URL}
        log_me("Logging in to {}".format(URL))
        r = s.post(URL, data=payload, verify=self.verify, headers=headers)

        # get the session ID and store it for all future API calls
        sessionid = s.cookies.get('sessionid')
        if not sessionid:
            log_me('No session ID, probably wrong user name / password')
        headers = {"X-CSRFToken": csrftoken,
                   "Cookie": f"csrftoken={csrftoken}; sessionid={sessionid}",
                   "Referer": URL
                   }

        return headers

    def get_token(self):
        change_token = "/api/v1/changeToken/"  # if you already have a token, use this url
        new_token = "/api/v1/getToken/"  # if you have never generated a token, use this url
        data = dict(username=self.account, password=self.password)
        response = requests.post(self.host + new_token, data=data)
        api_token = response.text
        if api_token == "EXISTING":
            response = requests.post(self.host + change_token, data=data)
            api_token = response.text
        return api_token

    # The generic_api_post method posts a request to Alation and if necessary checks the status
    def generic_api_post(self, api, params=None, body=None, official=False, raw_data=None):
        if official:
            headers_final = dict(token=self.token)
        else:
            headers_final = self.headers
            headers_final['Referer'] = self.host + api
        if raw_data:
          r = requests.post(self.host + api, data=raw_data, params=params, headers=headers_final)
        else:
          r = requests.post(self.host + api, json=body, params=params, headers=headers_final)

        if r.status_code:
            r_parsed = r.json()
            # do we need to ask the job status API for help?
            if 'job_id' in r_parsed:
                params = dict(id=r_parsed['job_id'])
                url_job = "/api/v1/bulk_metadata/job/"
                # Let's wait for the job to finish
                while (True):
                    time.sleep(10)
                    print(f"Waiting for job {params['id']}")
                    status = self.generic_api_get(api=url_job, params=params, official=True)
                    if not 'status' in status:
                      break
                    if status['status'] != 'running':
                        objects = status['result']
                        # if objects:
                        #     # for error in error_objects:
                        #     print(objects)
                        # else:
                        #     #print(status)
                        #     pass
                        break
                r_parsed = status
            return r_parsed
        else:
            return r.content

    # The generic_api_put method posts a request to Alation and if necessary checks the status
    def generic_api_put(self, api, params=None, body=None, official=False):
        if official:
            headers_final = dict(token=self.token)
        else:
            headers_final = self.headers
            headers_final['Referer'] = self.host + api
        r = requests.put(self.host + api, json=body, params=params, headers=headers_final, verify=self.verify)
        return r.content

    # The generic_api_patch method posts a request to Alation and if necessary checks the status
    def generic_api_patch(self, api, params=None, body=None):
        r = requests.patch(self.host + api, json=body, params=params, headers=self.headers, verify=self.verify)
        return r.content

    # The generic_api_get implements a REST get, with API token if official or Cookie if not.
    # If the callers sends header, it needs to contain API or cookie
    def generic_api_get(self, api, headers=None, params=None, official=False):
        if headers:
            # caller has supplied the headers
            headers_final = headers
        else:
            if official:
                headers_final = dict(token=self.token)
            else:
                headers_final = self.headers
                headers_final['Referer'] = self.host + api
        r = requests.get(self.host + api, headers=headers_final, params=params, verify=self.verify)
        if r.status_code in [200, 201]:
            try:
                return r.json()
            except:
                return r.content # for LogicalMetadata API which does not use standard JSON
        else:
            return r.content

    # The generic_api_get implements a REST get, with API token if official or Cookie if not.
    # If the callers sends header, it needs to contain API or cookie
    def raw_api_get(self, api, headers=None, params=None, official=False):
        if headers:
            # caller has supplied the headers
            headers_final = headers
        else:
            if official:
                headers_final = dict(token=self.token)
            else:
                headers_final = self.headers
                headers_final['Referer'] = self.host + api
        return requests.get(self.host + api, headers=headers_final, params=params, verify=self.verify)


Let's create our first AlationInstance object:

In [ ]:
def log_me(text):
  print(text)

url = 'http://ec2-54-149-165-141.us-west-2.compute.amazonaws.com'
user = 'matthias@alation.com'
alation = AlationInstance(url, user,'password')
h = alation.get_headers()

Logging in to http://ec2-54-149-165-141.us-west-2.compute.amazonaws.com/login/


## Downloading a data dictionary


In [15]:
body = dict(otype="schema", oid=2, format="csv")
alation.generic_api_post(f'/data/download_dict/', body=body)

Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249
Waiting for job 249


{'msg': 'Job finished in 29.524252 seconds at 2020-11-17 17:19:50.004685+00:00',
 'result': None,
 'status': 'successful'}

On the server, we use `ls -l /opt/alation/alation/opt/alation/site/downloads/data_dictionary/` to find out the file name. Then, we can download the file like this:

In [ ]:
filename = 'schema_1_14_2020-11-16T03-23-57-090905.csv'
print(f"{url}/download/data_dictionary/{filename}/")

http://ec2-54-149-165-141.us-west-2.compute.amazonaws.com/download/data_dictionary/schema_1_14_2020-11-16T03-23-57-090905.csv/


In [ ]:
r = requests.get(f"{url}/download/data_dictionary/{filename}/", headers=h)
csv_raw = r.text[3:]

Alternatively, we can use a Linux command on our Mac:

```rsync -av ec2-user@ec2-52-25-253-160.us-west-2.compute.amazonaws.com:/opt/alation/alation/opt/alation/site/downloads/ .```


In [ ]:
csv_raw

'key,title,description,user actions,update frequency,primary feed mechanism,health description,anything:article,anything:attribute,anything:data,anything:groupprofile,anything:schema,anything:table,anything:user,security classification,steward:groupprofile,steward:user,last compliance review,associated feed jobs,quality thresholds,requires risk assessment,origination system type,uniqueness score,health classification,quality rules,least frequent patterns,sensitive category,associated regulation,least frequent values,range of values,compliance review notes,most frequent values,most frequent patterns,contains restricted data,compliance steward:groupprofile,compliance steward:user,related policy:article,related risk assessments,contains sensitive data,domain:article,sensitive indicator,technical steward:groupprofile,technical steward:user,quality score,business steward:groupprofile,business steward:user\r\nbank,Banksy Schema Title,"<div>\n <p>\n  An important description\n </p>\n <p>\n  w

In [ ]:
import io
f = io.StringIO(csv_raw)
df = pd.read_csv(f)
df['key'] = df.key.apply(lambda t: f"1.{t}")
df.index = df['key']
df

,key,title,description,user actions,update frequency,primary feed mechanism,health description,anything:article,anything:attribute,anything:data,anything:groupprofile,anything:schema,anything:table,anything:user,security classification,steward:groupprofile,steward:user,last compliance review,associated feed jobs,quality thresholds,requires risk assessment,origination system type,uniqueness score,health classification,quality rules,least frequent patterns,sensitive category,associated regulation,least frequent values,range of values,compliance review notes,most frequent values,most frequent patterns,contains restricted data,compliance steward:groupprofile,compliance steward:user,related policy:article,related risk assessments,contains sensitive data,domain:article,sensitive indicator,technical steward:groupprofile,technical steward:user,quality score,business steward:groupprofile,business steward:user
key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.bank,1.bank,Banksy Schema Title,<div>\n <p>\n An important description\n </p>...,NaN,NaN,ETL Tool,<p>poor health</p>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,catalog admins,NaN,NaN,NaN,NaN,NaN,Staged Files,NaN,Red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matthias@alation.com,NaN,NaN,NaN,NaN,NaN,NaN,matthias@alation.com,<p>quality score seems to be here</p>,chief catalog steward,matthias@alation.com;antonio.fernandez@alation...
1.bank.profile_stats,1.bank.profile_stats,NaN,<p>\n No description available at 2020-11-10 0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.bank.test,1.bank.test,NaN,<p>\n No description available at 2020-11-10 0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.bank.account_detail,1.bank.account_detail,NaN,<p>\n No description available at 2020-11-10 0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.bank.trade_transactions_rms,1.bank.trade_transactions_rms,NaN,<p>\n No description available at 2020-11-10 0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.bank.trade_trxn.exercisedintosourcesystemid,1.bank.trade_trxn.exercisedintosourcesystemid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.bank.trade_type_summary.tradedate_yr,1.bank.trade_type_summary.tradedate_yr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.bank.trade_type_summary.tradedate_mth,1.bank.trade_type_summary.tradedate_mth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
my_row = dict(df.loc["1.bank"])

keys_to_delete = []

for k, v in my_row.items():
  if pd.isnull(v):
    keys_to_delete.append(k)

for k in keys_to_delete:
    del my_row[k]

my_row

{'business steward:groupprofile': 'chief catalog steward',
 'business steward:user': 'matthias@alation.com;antonio.fernandez@alation.com',
 'compliance steward:user': 'matthias@alation.com',
 'description': '<div>\n <p>\n  An important description\n </p>\n <p>\n  with a picture\n </p>\n <p>\n  <img class="fr-fic fr-dib" src="/media/image_bank/2020-11-10-11-56-19-671061-00-00-b71e7e9f-569f-4566-8540-ae4459211f75.png" style="width: 300px;">\n </p>\n</div>',
 'health classification': 'Red',
 'health description': '<p>poor health</p>',
 'key': '1.bank',
 'origination system type': 'Staged Files',
 'primary feed mechanism': 'ETL Tool',
 'quality score': '<p>quality score seems to be here</p>',
 'steward:groupprofile': 'catalog admins',
 'technical steward:user': 'matthias@alation.com',
 'title': 'Banksy Schema Title'}

In [ ]:
from collections import defaultdict

list_of_objects = []
new_dict = defaultdict(list)

for k, v in my_row.items():
  if ":" in k:
    list_of_objects.append(k)
    components = k.split(":")
    my_obj = components[0]
    my_values = v.split(";")
    for v0 in my_values:
      new_dict[my_obj].append(dict(type=components[1], key=v0))

for k in list_of_objects:
    del my_row[k]

d = dict(**my_row, **new_dict)
d

{'business steward': [{'key': 'chief catalog steward', 'type': 'groupprofile'},
  {'key': 'matthias@alation.com', 'type': 'user'},
  {'key': 'antonio.fernandez@alation.com', 'type': 'user'}],
 'compliance steward': [{'key': 'matthias@alation.com', 'type': 'user'}],
 'description': '<div>\n <p>\n  An important description\n </p>\n <p>\n  with a picture\n </p>\n <p>\n  <img class="fr-fic fr-dib" src="/media/image_bank/2020-11-10-11-56-19-671061-00-00-b71e7e9f-569f-4566-8540-ae4459211f75.png" style="width: 300px;">\n </p>\n</div>',
 'health classification': 'Red',
 'health description': '<p>poor health</p>',
 'key': '1.bank',
 'origination system type': 'Staged Files',
 'primary feed mechanism': 'ETL Tool',
 'quality score': '<p>quality score seems to be here</p>',
 'steward': [{'key': 'catalog admins', 'type': 'groupprofile'}],
 'technical steward': [{'key': 'matthias@alation.com', 'type': 'user'}],
 'title': 'Banksy Schema Title'}

In [ ]:
d['key'] = "2.census"
d['title'] = "A flashy title for the census schema"
del d['quality score']
del d['health description']
d

{'business steward': [{'key': 'chief catalog steward', 'type': 'groupprofile'},
  {'key': 'matthias@alation.com', 'type': 'user'},
  {'key': 'antonio.fernandez@alation.com', 'type': 'user'}],
 'compliance steward': [{'key': 'matthias@alation.com', 'type': 'user'}],
 'description': '<div>\n <p>\n  An important description\n </p>\n <p>\n  with a picture\n </p>\n <p>\n  <img class="fr-fic fr-dib" src="/media/image_bank/2020-11-10-11-56-19-671061-00-00-b71e7e9f-569f-4566-8540-ae4459211f75.png" style="width: 300px;">\n </p>\n</div>',
 'health classification': 'Red',
 'key': '2.census',
 'origination system type': 'Staged Files',
 'primary feed mechanism': 'ETL Tool',
 'steward': [{'key': 'catalog admins', 'type': 'groupprofile'}],
 'technical steward': [{'key': 'matthias@alation.com', 'type': 'user'}],
 'title': 'A flashy title for the census schema'}

In [ ]:
import json
json_row = json.dumps(d)
json_row

'{"key": "2.census", "title": "A flashy title for the census schema", "description": "<div>\\n <p>\\n  An important description\\n </p>\\n <p>\\n  with a picture\\n </p>\\n <p>\\n  <img class=\\"fr-fic fr-dib\\" src=\\"/media/image_bank/2020-11-10-11-56-19-671061-00-00-b71e7e9f-569f-4566-8540-ae4459211f75.png\\" style=\\"width: 300px;\\">\\n </p>\\n</div>", "primary feed mechanism": "ETL Tool", "origination system type": "Staged Files", "health classification": "Red", "steward": [{"type": "groupprofile", "key": "catalog admins"}], "compliance steward": [{"type": "user", "key": "matthias@alation.com"}], "technical steward": [{"type": "user", "key": "matthias@alation.com"}], "business steward": [{"type": "groupprofile", "key": "chief catalog steward"}, {"type": "user", "key": "matthias@alation.com"}, {"type": "user", "key": "antonio.fernandez@alation.com"}]}'

In [ ]:
params = dict(replace_values=True)
alation.generic_api_post("/api/v1/bulk_metadata/custom_fields/default/mixed", raw_data=json_row, params=params)

{'error': '',
 'error_objects': [],
 'new_objects': 0,
 'number_received': 1,
 'updated_objects': 1}

In [16]:
alation.generic_api_get("/integration/v2/dataflow/")

{'dataflow_objects': [], 'paths': []}